In [27]:
"""
Amstrad CPC Mode 0 Tile Converter (CORRECTED)
==============================================
Converts .bin hex files to 128 individual 8x8 pixel tiles (Mode 0 format)
Output PNGs are doubled horizontally to correct for rectangular pixels.

CORRECT Mode 0 Encoding:
- 2 pixels per byte (4 bits per pixel = 16 colors from hardware palette)
- Each tile: 8×8 pixels = 8 rows × 4 bytes per row = 32 bytes total
- 128 tiles = 4096 bytes (0x1000) total
- Bit interleaving: For each byte, the 2 pixels use bits:
  * Pixel 0 (left):  bits 7,6,5,4
  * Pixel 1 (right): bits 3,2,1,0
- Output PNGs are 16×8 (doubled horizontally) to correct for rectangular CPC pixels

File Layout:
- Total: 128 tiles in sequence
- Tiles numbered 0x00 to 0x7F (0-127 decimal)
- Each tile at offset: tile_number × 32 bytes

Usage:
1. Adjust PALETTE_COLORS below to match your game's palette
2. Set input_file to your .bin file path (should be 4096 bytes)
3. Run to generate tile_00.png through tile_7F.png
4. Compare output with reference screenshots to verify colors
"""

import numpy as np
from PIL import Image
import os

# ============================================================================
# PALETTE CONFIGURATION - MODIFY THESE VALUES
# ============================================================================
PALETTE_COLORS = [
    (0x00, 0x00, 0x00),  # Index 0  - Black (Corrected)
    (0x80, 0x80, 0x80),  # Index 1  - Grey (Corrected)
    (0xFF, 0xFF, 0xFF),  # Index 2  - white (Corrected)
    (0x80, 0x80, 0x00),  # Index 3  - Yellow (Corrected)
    (0xFF, 0xFF, 0x00),  # Index 4  - Bright Yellow (Corrected)
    (0x00, 0x80, 0x00),  # Index 5  - Green (Corrected)
    (0x00, 0xFF, 0x00),  # Index 6  - Bright Green (Corrected)
    (0x00, 0x80, 0x80),  # Index 7  - Cyan (Corrected)
    (0x00, 0xFF, 0xFF),  # Index 8  - Bright Cyan (Corrected)
    (0x00, 0x00, 0x80),  # Index 9  - Blue (Corrected)
    (0xFF, 0x80, 0x80),  # Index 10 - Pink (Corrected)
    (0xFF, 0xFF, 0x80),  # Index 11 - Pastle Yellow (Corrected)
    (0x80, 0x80, 0xFF),  # Index 12 - Pastle Blue (Corrected)
    (0xFF, 0x80, 0x00),  # Index 13 - Orange (Corrected)
    (0x80, 0x00, 0x00),  # Index 14 - Red (Corrected)
    (0xFF, 0x00, 0x00),  # Index 15 - Bright Red (Corrected)
]


# ============================================================================
# CONFIGURATION
# ============================================================================
input_file = "eagles-amstrad"  # Your input .bin file
output_dir = "amstrad_tiles_output"  # Directory for PNG tiles
TILE_WIDTH = 8   # Pixels per tile (Mode 0)
TILE_HEIGHT = 8  # Pixels per tile
BYTES_PER_TILE = 32  # 8 rows × 4 bytes per row
TOTAL_TILES = 128  # Number of tiles to extract

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================
def decode_mode0_byte(byte_value):
    """
    Decode a single CPC Mode 0 byte into 2 pixels.

    CPC Mode 0 bit layout:
    Bit 7 = P0 bit 0
    Bit 6 = P1 bit 0
    Bit 5 = P0 bit 2
    Bit 4 = P1 bit 2
    Bit 3 = P0 bit 1
    Bit 2 = P1 bit 1
    Bit 1 = P0 bit 3
    Bit 0 = P1 bit 3
    """

    p0 = (
        ((byte_value >> 7) & 1) << 0 |
        ((byte_value >> 3) & 1) << 1 |
        ((byte_value >> 5) & 1) << 2 |
        ((byte_value >> 1) & 1) << 3
    )

    p1 = (
        ((byte_value >> 6) & 1) << 0 |
        ((byte_value >> 2) & 1) << 1 |
        ((byte_value >> 4) & 1) << 2 |
        ((byte_value >> 0) & 1) << 3
    )

    return [p0, p1]

# def decode_mode0_byte(byte_value):
#     """
#     Decode a single byte into 2 pixels (Mode 0 format).
    
#     In Mode 0, each byte encodes 2 pixels with 4 bits per pixel.
#     Bit layout: [7654][3210]
#     - Left pixel:  bits 7,6,5,4
#     - Right pixel: bits 3,2,1,0
    
#     Args:
#         byte_value: Single byte (0-255)
    
#     Returns:
#         List of 2 pixel values (palette indices 0-15)
#     """
#     left_pixel = (byte_value >> 4) & 0xF  # Upper 4 bits
#     right_pixel = byte_value & 0xF         # Lower 4 bits
#     return [left_pixel, right_pixel]


def decode_mode0_tile(tile_data):
    """
    Decode a 32-byte tile into an 8×8 pixel array.
    
    Amstrad CPC Mode 0 storage:
    - Each scanline = 4 bytes (8 pixels ÷ 2 pixels/byte)
    - 8 scanlines = 32 bytes total
    - No interleaving, bytes are sequential
    
    Args:
        tile_data: 32 bytes of tile data
    
    Returns:
        8×8 numpy array of palette indices
    """
    if len(tile_data) != BYTES_PER_TILE:
        raise ValueError(f"Tile data must be {BYTES_PER_TILE} bytes, got {len(tile_data)}")
    
    tile = np.zeros((TILE_HEIGHT, TILE_WIDTH), dtype=np.uint8)
    
    for row in range(TILE_HEIGHT):
        # Each row uses 4 consecutive bytes
        byte_offset = row * 4
        
        # Decode 4 bytes = 8 pixels
        for byte_idx in range(4):
            byte_val = tile_data[byte_offset + byte_idx]
            pixels = decode_mode0_byte(byte_val)
            
            # Place 2 pixels at correct position
            tile[row, byte_idx * 2] = pixels[0]
            tile[row, byte_idx * 2 + 1] = pixels[1]
    
    return tile


def tile_to_image(tile_array, palette):
    """
    Convert a tile array to a PIL Image with horizontal doubling.
    
    Args:
        tile_array: 8×8 array of palette indices
        palette: List of RGB tuples
    
    Returns:
        PIL Image (16×8 to account for rectangular pixels)
    """
    height, width = tile_array.shape
    
    # Create RGB image with horizontal doubling
    img = Image.new('RGB', (width * 2, height))
    pixels = img.load()
    
    for y in range(height):
        for x in range(width):
            color_index = tile_array[y, x]
            if color_index >= len(palette):
                print(f"Warning: Color index {color_index} out of palette range, using black")
                color = (0, 0, 0)
            else:
                color = palette[color_index]
            
            # Double horizontally for rectangular pixel correction
            pixels[x * 2, y] = color
            pixels[x * 2 + 1, y] = color
    
    return img


def verify_test_tile():
    """
    Verify the decoder using the provided test data.
    Expected hex: 00000000 41C3C300 8282C382 41414100 00000000 C3004141 C3828282 41004141
    """
    test_hex = "00000000 41C3C300 8282C382 41414100 00000000 C3004141 C3828282 41004141"
    test_bytes = bytes.fromhex(test_hex.replace(" ", ""))
    
    print("Test Tile Verification")
    print("=" * 60)
    print(f"Hex data: {test_hex}")
    print(f"Bytes: {len(test_bytes)} (should be 32 for 1 tile)")
    
    if len(test_bytes) != 32:
        print(f"WARNING: Expected 32 bytes, got {len(test_bytes)}")
        return
    
    tile = decode_mode0_tile(test_bytes)
    print("\nDecoded tile (palette indices):")
    for i, row in enumerate(tile):
        print(f"Row {i}: {' '.join(f'{val:X}' for val in row)}")
    
    # Save test tile
    test_img = tile_to_image(tile, PALETTE_COLORS)
    test_output = "test_tile.png"
    test_img.save(test_output)
    print(f"\nTest tile saved as: {test_output}")
    print("Compare this with your Stone.png to verify colors are correct.")
    print()


def extract_tiles(input_file, output_dir, palette):
    """
    Extract all 128 tiles from a .bin file.
    
    Args:
        input_file: Path to .bin file
        output_dir: Directory to save PNG files
        palette: List of RGB tuples
    
    Returns:
        List of decoded tile arrays for composite image creation
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Read binary file
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' not found!")
        print("Please update the 'input_file' variable with your .bin file path.")
        return []
    
    with open(input_file, 'rb') as f:
        data = f.read()
    
    expected_size = TOTAL_TILES * BYTES_PER_TILE
    if len(data) < expected_size:
        print(f"Warning: File is {len(data)} bytes, expected at least {expected_size} bytes")
        print(f"Will extract {len(data) // BYTES_PER_TILE} tiles instead of {TOTAL_TILES}")
        actual_tiles = len(data) // BYTES_PER_TILE
    else:
        actual_tiles = TOTAL_TILES
    
    print(f"Extracting {actual_tiles} tiles from {input_file}...")
    print(f"Output directory: {output_dir}")
    print()
    
    tile_arrays = []
    
    for tile_num in range(actual_tiles):
        # Extract 32 bytes for this tile
        offset = tile_num * BYTES_PER_TILE
        tile_data = data[offset:offset + BYTES_PER_TILE]
        
        # Decode tile
        tile_array = decode_mode0_tile(tile_data)
        tile_arrays.append(tile_array)
        
        # Convert to image
        img = tile_to_image(tile_array, palette)
        
        # Save with hex filename
        filename = f"tile_{tile_num:02X}.png"
        filepath = os.path.join(output_dir, filename)
        img.save(filepath)
        
        if (tile_num + 1) % 16 == 0:
            print(f"Processed {tile_num + 1}/{actual_tiles} tiles...")
    
    print(f"\nComplete! {actual_tiles} tiles saved to {output_dir}/")
    print(f"Files: tile_00.png through tile_{actual_tiles-1:02X}.png")
    
    return tile_arrays


def create_composite_image(tile_arrays, palette, output_file="tileset_composite.png"):
    """
    Create a composite image showing all tiles in an 8×16 grid.
    
    Args:
        tile_arrays: List of 128 8×8 tile arrays
        palette: List of RGB tuples
        output_file: Output filename
    """
    if len(tile_arrays) != 128:
        print(f"Warning: Expected 128 tiles, got {len(tile_arrays)}")
        return
    
    # Layout: 8 rows × 16 columns of tiles
    # Each tile is 8×8 pixels, doubled horizontally = 16×8 pixels
    rows = 8
    cols = 16
    tile_width_display = 16  # Doubled horizontally
    tile_height_display = 8
    
    composite_width = cols * tile_width_display
    composite_height = rows * tile_height_display
    
    composite = Image.new('RGB', (composite_width, composite_height))
    
    for idx, tile_array in enumerate(tile_arrays):
        row = idx // cols
        col = idx % cols
        
        # Convert tile to image
        tile_img = tile_to_image(tile_array, palette)
        
        # Paste into composite
        x_offset = col * tile_width_display
        y_offset = row * tile_height_display
        composite.paste(tile_img, (x_offset, y_offset))
    
    composite.save(output_file)
    print(f"\nComposite image saved as: {output_file}")
    print(f"Dimensions: {composite_width}×{composite_height} pixels")
    print(f"Layout: 8 rows × 16 columns = 128 tiles")


def show_palette_reference():
    """Display the current palette for reference."""
    print("Current Palette")
    print("=" * 60)
    for i, color in enumerate(PALETTE_COLORS):
        r, g, b = color
        print(f"Index {i:2d} (0x{i:X}): RGB({r:3d}, {g:3d}, {b:3d}) - #{r:02X}{g:02X}{b:02X}")
    print()


def create_palette_swatch():
    """Create a visual swatch of all palette colors."""
    swatch_width = 32
    swatch_height = 32
    cols = 4
    rows = (len(PALETTE_COLORS) + cols - 1) // cols
    
    img = Image.new('RGB', (swatch_width * cols, swatch_height * rows))
    pixels = img.load()
    
    for idx, color in enumerate(PALETTE_COLORS):
        col = idx % cols
        row = idx // cols
        
        for y in range(swatch_height):
            for x in range(swatch_width):
                pixels[col * swatch_width + x, row * swatch_height + y] = color
    
    img.save("palette_swatch.png")
    print("Palette swatch saved as: palette_swatch.png")
    print()


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("=" * 60)
    print("Amstrad CPC Mode 0 Tile Converter (CORRECTED)")
    print("=" * 60)
    print("Format: 32 bytes per tile (8 rows × 4 bytes)")
    print("File size: 4096 bytes for 128 tiles")
    print()
    
    # Show current palette
    show_palette_reference()
    
    # Create palette swatch
    create_palette_swatch()
    
    # Run verification test
    verify_test_tile()
    
    # Extract all tiles
    tile_arrays = extract_tiles(input_file, output_dir, PALETTE_COLORS)
    
    # Create composite image
    if tile_arrays:
        create_composite_image(tile_arrays, PALETTE_COLORS)
    
    print("\n" + "=" * 60)
    print("NEXT STEPS:")
    print("=" * 60)
    print("1. Check test_tile.png against your Stone.png reference")
    print("2. Check tileset_composite.png to see all 128 tiles at once")
    print("3. If colors are wrong, modify PALETTE_COLORS at top of script")
    print("4. Use palette_swatch.png to see all 16 colors")
    print("5. Re-run script after adjusting palette")
    print("6. Once colors are correct, all tiles will be properly colored")
    print("=" * 60)


Amstrad CPC Mode 0 Tile Converter (CORRECTED)
Format: 32 bytes per tile (8 rows × 4 bytes)
File size: 4096 bytes for 128 tiles

Current Palette
Index  0 (0x0): RGB(  0,   0,   0) - #000000
Index  1 (0x1): RGB(128, 128, 128) - #808080
Index  2 (0x2): RGB(255, 255, 255) - #FFFFFF
Index  3 (0x3): RGB(128, 128,   0) - #808000
Index  4 (0x4): RGB(255, 255,   0) - #FFFF00
Index  5 (0x5): RGB(  0, 128,   0) - #008000
Index  6 (0x6): RGB(  0, 255,   0) - #00FF00
Index  7 (0x7): RGB(  0, 128, 128) - #008080
Index  8 (0x8): RGB(  0, 255, 255) - #00FFFF
Index  9 (0x9): RGB(  0,   0, 128) - #000080
Index 10 (0xA): RGB(255, 128, 128) - #FF8080
Index 11 (0xB): RGB(255, 255, 128) - #FFFF80
Index 12 (0xC): RGB(128, 128, 255) - #8080FF
Index 13 (0xD): RGB(255, 128,   0) - #FF8000
Index 14 (0xE): RGB(128,   0,   0) - #800000
Index 15 (0xF): RGB(255,   0,   0) - #FF0000

Palette swatch saved as: palette_swatch.png

Test Tile Verification
Hex data: 00000000 41C3C300 8282C382 41414100 00000000 C3004141 C38